

# 1. Install packages

In [ ]:
!pip install geopandas --upgrade pip
!pip install geopy --upgrade pip

In [ ]:
import geopy
import geopandas
import pandas as pd
!pip install osmnx

# 2. Load datasets

# # Opioid Data at the Zip Code Level
There are data for deaths, ED visits, hospitalizations, and prescriptions

3/4 - Need to merge the dataframes.

In [ ]:
# 4!!

# import opioid death data
opdeath18_data = pd.read_csv("https://www.dropbox.com/s/iblpnv70nhsyrok/Death_SanFrancisco_AnyOpioid_2018.csv?dl=1")
    
opdeath18_data.rename(columns = {'San Francisco Deaths - 2018' : 'Zip Code',
                            'Unnamed: 1': 'Death Rates', 
                             'Unnamed: 2': 'Death 95% LCL', 
                             'Unnamed: 3' : 'Death 95% UCL'}, inplace=True)
#preview the data
opdeath18_data.head()

In [ ]:
#cut out the top two rows bc they are metadata
opdeath18cln = opdeath18_data.drop(opdeath18_data.index[[0,1,29,30,31,32]])

In [ ]:
print(opdeath18cln)

In [ ]:
# 5!! CA Opioid data

#Import the opioid ED Visit 2018 data

oped18_data = pd.read_csv("https://www.dropbox.com/s/1a8v10nnam0788j/EdVisit_SanFrancisco_AnyOpioid_2018.csv?dl=1")

oped18_data.rename(columns = {'San Francisco ED Visits - 2018' : 'Zip Code',
                            'Unnamed: 1': 'ED Rates', 
                             'Unnamed: 2': 'ED 95% LCL', 
                             'Unnamed: 3' : 'ED 95% UCL'}, inplace=True)

#preview the data
oped18_data.head()

In [ ]:
#drop rows 0 and 1, which are metadata 
oped18cln = oped18_data.drop(oped18_data.index[[0,1,29,30,31,32]])

In [ ]:
oped18cln.describe()

In [ ]:
# 6!!

#Import the opioid Hospitilization 2018 data

Ophosp18_data = pd.read_csv("https://www.dropbox.com/s/0rcfdjy3lg40gqq/Hospit_SanFrancisco_AnyOpioid_2018.csv?dl=1")
Ophosp18_data.rename(columns = {'San Francisco Hospitalizations - 2018': 'Zip Code',  
                            'Unnamed: 1': 'Hosp Rates', 
                             'Unnamed: 2': 'Hosp 95% LCL', 
                             'Unnamed: 3' : 'Hosp 95% UCL'}, inplace=True)

#preview the data
Ophosp18_data.head()

# 3. Geocode the addresses to facilities


In [ ]:
locator = geopy.geocoders.Nominatim(user_agent='myGeocoder')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ryglaws/255_Final/master/Facilities_Folium_4.16.csv')
df.head()

In [ ]:
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['geocode'] = df['location'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['geocode'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [ ]:
df.isnull()

In [ ]:
# Drop rows with NaN or missing value in all columns
facilities = df.dropna()

In [ ]:
facilities.head(140)

In [ ]:
# Isolate the drugstores

drug_store = facilities.loc[facilities.facility_type=='Drug Store']
drug_store.head(140)

In [ ]:
# Isolate the healthcare facilities

health_care=facilities.loc[facilities.facility_type!='Drug Store']
health_care.head(140)

# 4. Sort facilities by zip code


In [ ]:
import numpy as np
# Set zipcode type to string (folium)
facilities['zip'] = facilities['zip'].astype('str')
# get the mean value across all data points
zipcode_data = facilities.groupby('zip').aggregate(np.mean)
zipcode_data.reset_index(inplace = True)

In [ ]:
# count number of faclities grouped by zipcode
facilities['count'] = 1
temp = facilities.groupby('zip').sum()
temp.reset_index(inplace = True)
temp = temp[['zip', 'count']]
zipcode_data = pd.merge(zipcode_data, temp, on='zip')
# drop count from org dataset
facilities.drop(['count'], axis = 1, inplace = True)

In [ ]:
temp.head(55)

In [ ]:
temp.dtypes

# 4. Map the facilities


In [1]:
! pip install folium
import folium

You are using pip version 9.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
map1 = folium.Map(
    location=[37.764,-122.4194],
    tiles='cartodbpositron',
    zoom_start=12,
)
facilities.apply(lambda row:folium.Marker(location=[row["latitude"], row["longitude"]], 
popup=(row["facility_name"])).add_to(map1), axis=1)
map1

In [ ]:

from folium.plugins import FastMarkerCluster

folium_map = folium.Map(
    location=[37.764,-122.4194],
    tiles='cartodbpositron',
    zoom_start=12,
)

FastMarkerCluster(data=list(zip(facilities['latitude'].values, facilities['longitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

# 5. Add SF Zipcodes

In [ ]:
import json
from folium import plugins
with open('BayAreaZIPCodes.geojson') as Json:
    data = json.load(Json)
zipson = data

In [ ]:
print(zipson)

In [ ]:
# remove ZIP codes not in geo data
geozips = []
for i in range(len(zipson['features'])):
    if zipson['features'][i]['properties']['zip'] in list(zipcode_data['zip'].unique()):
        geozips.append(zipson['features'][i])
# creating new JSON object
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips
# save uodated JSON object
open("cleaned_geodata.json", "w").write(json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': ')))

In [ ]:
zipcode_data.head()

In [ ]:
from folium.plugins import MarkerCluster
    # read updated geo data
sf_geo = "cleaned_geodata.json"

    # Initialize Folium Map with Seattle latitude and longitude
    #m = folium.Map(location=[37.764,-122.4194], zoom_start=12,
                   #detect_retina=True, control_scale=False)
    
choromap = folium.Map(location=[37.764,-122.4194],
                      tiles='cartodbpositron',
                      zoom_start=12)

    # Create choropleth map
choromap.choropleth(
    geo_data=sf_geo,
    name='choropleth',
    data=zipcode_data,
    # col: feature of interest
    columns=['zip', 'count'],
    key_on='feature.properties.zip',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Facilities by Zipcode')

    #folium.LayerControl().add_to(choromap)
    
    #Add marker clusters
FastMarkerCluster(data=list(zip(facilities['latitude'].values, facilities['longitude'].values))).add_to(choromap)
folium.LayerControl().add_to(choromap)
 
# now, all the markers should be directed to this objects as follows: folium.Marker(...).add_to(marker_cluster)

    # Save map based on feature of interest
    #m.save(col + '.html')

    #return m
choromap

In [ ]:
import numpy as np

# Set zipcode type to string (folium)
drug_store['zip'] = drug_store['zip'].astype('str')
# get the mean value across all data points
zipcode_data2 = drug_store.groupby('zip').aggregate(np.mean)
zipcode_data2.reset_index(inplace = True)

In [ ]:
# count number of drug stores grouped by zipcode
drug_store['count'] = 1
temp2 = drug_store.groupby('zip').sum()
temp2.reset_index(inplace = True)
temp2 = temp2[['zip', 'count']]
zipcode_data2 = pd.merge(zipcode_data2, temp2, on='zip')
# drop count from org dataset
drug_store.drop(['count'], axis = 1, inplace = True)

In [ ]:
# Map of drug stores

choromap2 = folium.Map(location=[37.764,-122.4194],
                      tiles='cartodbpositron',
                      zoom_start=12)

    # Create choropleth map
choromap2.choropleth(
    geo_data=sf_geo,
    name='choropleth',
    data=zipcode_data2,
    # col: feature of interest
    columns=['zip', 'count'],
    key_on='feature.properties.zip',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Drug Stores by Zipcode')# Can we set the polygons to zip codes?

cluster_map = folium.Map(
    location=[37.764,-122.4194],
    tiles='cartodbpositron',
    zoom_start=12,
)

FastMarkerCluster(data=list(zip(drug_store['latitude'].values, drug_store['longitude'].values))).add_to(choromap2)
folium.LayerControl().add_to(choromap2)

choromap2

In [ ]:
import numpy as np

# Set zipcode type to string (folium)
health_care['zip'] = health_care['zip'].astype('str')
# get the mean value across all data points
zipcode_data3 = health_care.groupby('zip').aggregate(np.mean)
zipcode_data3.reset_index(inplace = True)

In [ ]:
# count number of health centers grouped by zipcode
health_care['count'] = 1
temp3 = health_care.groupby('zip').sum()
temp3.reset_index(inplace = True)
temp3 = temp3[['zip', 'count']]
zipcode_data3 = pd.merge(zipcode_data3, temp3, on='zip')
# drop count from org dataset
health_care.drop(['count'], axis = 1, inplace = True)

In [ ]:
#Map of Health Centers

choromap3 = folium.Map(location=[37.764,-122.4194],
                      tiles='cartodbpositron',
                      zoom_start=12)

    # Create choropleth map
choromap3.choropleth(
    geo_data=sf_geo,
    name='choropleth',
    data=zipcode_data3,
    # col: feature of interest
    columns=['zip', 'count'],
    key_on='feature.properties.zip',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Healthcare Facilities by Zipcode')# Can we set the polygons to zip codes?

cluster_map = folium.Map(
    location=[37.764,-122.4194],
    tiles='cartodbpositron',
    zoom_start=12,
)

FastMarkerCluster(data=list(zip(health_care['latitude'].values, health_care['longitude'].values))).add_to(choromap3)
folium.LayerControl().add_to(choromap3)

choromap3

In [ ]:
%%capture
!pip install plotly --upgrade

In [ ]:
import plotly
plotly.__version__

In [ ]:
import plotly.express as px

In [ ]:
# First try with a simple popup

map3 = folium.Map(
    location=[37.764,-122.4194],
    tiles='cartodbpositron',
    zoom_start=12,
)

tooltip = 'Click me!'
    
health_care.apply(lambda row:folium.Marker(
    location=[row["latitude"], row["longitude"]],
    popup=(row['facility_name'],
           row['location']),
    icon=folium.Icon(color='red'),
    tooltip=tooltip
).add_to(map3), axis=1)

drug_store.apply(lambda row:folium.Marker(
    location=[row["latitude"], row["longitude"]],
    popup=(row['facility_name'],  
           row['location']),
    icon=folium.Icon(color='green'),
    tooltip=tooltip
).add_to(map3), axis=1)

from folium.plugins import FloatImage
image_file = 'https://raw.githubusercontent.com/ryglaws/255_Final/master/Legend2.png'

FloatImage(image_file, bottom=2.5, left=85).add_to(map3)
map3

In [ ]:
map3.save(outfile='facilitiesmap.html')